This is a first attempt at establishing some form of electric cooking potential index. I have taken four datasets to identify large urban populations that are (1) using primarily biomass fuels (2) have access to electricity and (3) have sufficient income to purchase and use an induction stove, potentially with government or NGO assistance.

The four datasets I am using are:
1. The United Nations World Urbanization Prospects 2018 dataset on all cities with over 300000 residents (https://population.un.org/wup/Download/)
2. The WHO Household Energy database for proportions of residents using dirty fuels (by country) (https://www.who.int/data/gho/data/themes/air-pollution/who-household-energy-db)
3. The World Bank World Development Indicators for proportion of urban residents with access to electricity (by country) (https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators)
4. Data from Our World in Data's poverty explorer (https://ourworldindata.org/poverty) originally compiled from World Bank Income Data (https://pip.worldbank.org/home) to get the proportion of residents living in extreme poverty.

I combine these datasets into one dataframe containing:
1) the urban agglomeration (UA)
2) the UA's population
3) the proportion of the parent country's urban residents using dirty fuels
4) the proportion of the parent country's urban residents with electricty access
5) the proportion of the parent country's residents living in extreme poverty.

From these columns I create a population estimate by multiplying the dirty fuel, electricity, and not-extreme poverty proportions by the UA's population.

I can see several major issues with this first attempt.

First of all, the resident proportions I am using are by country, not by urban area. This will definitely lead to weird results where countries can have highly varied development (e.g. Shanghai is listed as one of the largest populations with dirty fuels, which seems unlikely given what I know about Shanghai). Ajay provided one dataset for income that might amend this.

Second, there is no information on the quality of electricity access. The robustness of a city's infrastructure will play a huge role in whether induction stoves are a scalable solution in the near term, so we have to find a way to incorporate this.

Third, I'm not really sure how to define a cutoff point for "able to afford an induction stove." This depends on the price of stoves, electricity prices, the size of potential subsidies, and income. Related is how willing people would actually be to use such stoves. If LPG is cheaper, then people will just use that instead (Ecuador is a good example).

Fourth, we don't have any information on potential for government cooperation (e.g. corruption indexes).

Fifth, just wondering about other factors I may be missing. e.g. environmental factors seem important. For example, electric stoves wouldn't work in Mongolia because the stoves are used for heating homes, in addition to cooking.

Sixth, there is probably higher overlap between dirty fuel/poor electricity/low income people. So we are likely overestimating populations currently.


Thinking in terms of (1) the potential health impacts of flipping a population to electricity and (2) the actual uptake scales and probabilities. (1) Seems much easier to measure (just the total dirty fuel population). Urban density, extent of fuel use, and size of clean fuel population would also matter in terms of total health effects. (2) Is difficult. It requires understanding electricity infrastructure/stability, government stability, financing, and political support, income and economics of the population, geography/environment, and a whole lot else. Phew.

Other observations:
1. We're missing out on small island countries. Might be reasonable to include these anyway though. Issues of justice/bias here, plus such countries could be interesting proving grounds for scaling up interventions.
2. Almost half the cities in this list are in India or China
3. Income data isn't consistent across years (some countries have more recent data than others). More of a reason to use the dataset Ajay suggested.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

In [19]:
# Load data
df_pop = pd.read_csv('../Data/WUP2018-F22-Cities_Over_300K_Annual.csv', encoding = 'unicode_escape', engine ='python')
df_fuel = pd.read_csv('../Data/fuel-use-by-country-2020.csv', encoding = 'unicode_escape', engine ='python')
df_elec = pd.read_csv('../Data/world-dev-indicators-electricity-urban.csv')
df_income = pd.read_csv('../Data/poverty-explorer.csv')

In [20]:
# Fix errors in fuel dataframe
df_fuel['Location'] = df_fuel['Location'].str.replace('Côte d\x92Ivoire', 'Côte d\'Ivoire')
df_fuel['Location'] = df_fuel['Location'].str.replace('Democratic People\'s Republic of Korea', 'Dem. People\'s Republic of Korea')
df_fuel['Location'] = df_fuel['Location'].str.replace('The former Yugoslav Republic of Macedonia', 'TFYR Macedonia')


# Select fuels considered dirty and consolidate into a "dirty fuel proportion"
mask1 = (df_fuel['IndicatorCode'] == 'PHE_HHAIR_PROP_POP_CATEGORY_FUELS')
mask2 = (df_fuel['Dim1'] == 'Urban')
mask3 = (df_fuel['Dim2'].isin(['Kerosene', 'Biomass', 'Charcoal', 'Coal']))
prop_biomass = df_fuel.loc[mask1 & mask2 & mask3, ['SpatialDimValueCode', 'FactValueNumeric']].groupby('SpatialDimValueCode').sum() / 100
prop_biomass.rename(columns={'FactValueNumeric': 'proportion dirty fuel'}, inplace=True)

codes = df_fuel.loc[:, ['SpatialDimValueCode', 'Location']].drop_duplicates().set_index('SpatialDimValueCode')
prop_biomass = prop_biomass.join(codes).reset_index().set_index('Location')

In [21]:
# Join dirty fuel proportion data to population data
df = df_pop.join(prop_biomass, on='Country or area')
df['2020'] = df['2020'].str.replace(' ', '').astype(int)
df['estimated population burning dirty fuels'] = np.round(df['2020'] * df['proportion dirty fuel'] * 1000).astype(int)
df.dropna(inplace=True, subset=['proportion dirty fuel'])

In [22]:
# Join electricity data to population data
prop_elec = df_elec.loc[:, ['Country Code', '2020 [YR2020]']].set_index('Country Code') / 100
df = df.join(prop_elec, on='SpatialDimValueCode')
df.rename(columns={'2020 [YR2020]': 'proportion electricity access'}, inplace=True)

In [23]:
# Fix errors in income data
df_income['Entity'] = df_income['Entity'].str.replace(' - urban', '')
df_income['Entity'] = df_income['Entity'].str.replace('Bolivia', 'Bolivia (Plurinational State of)')
df_income['Entity'] = df_income['Entity'].str.replace('Cote d\'Ivoire', 'Côte d\'Ivoire')
df_income['Entity'] = df_income['Entity'].str.replace('Democratic Republic of Congo', 'Democratic Republic of the Congo')
df_income['Entity'] = df_income['Entity'].str.replace('Iran', 'Iran (Islamic Republic of)')
df_income['Entity'] = df_income['Entity'].str.replace('Laos', 'Lao People\'s Democratic Republic')
df_income['Entity'] = df_income['Entity'].str.replace('Moldova', 'Republic of Moldova')
df_income['Entity'] = df_income['Entity'].str.replace('North Macedonia', 'TFYR Macedonia')
df_income['Entity'] = df_income['Entity'].str.replace('Russia', 'Russian Federation')
df_income['Entity'] = df_income['Entity'].str.replace('South Korea', 'Republic of Korea')
df_income['Entity'] = df_income['Entity'].str.replace('Syria', 'Syrian Arab Republic')
df_income['Entity'] = df_income['Entity'].str.replace('Tanzania', 'United Republic of Tanzania')
df_income['Entity'] = df_income['Entity'].str.replace('Venezuela', 'Venezuela (Bolivarian Republic of)')
df_income['Entity'] = df_income['Entity'].str.replace('Vietnam', 'Viet Nam')

# Join income data to population data
index = df_income.sort_values(by='Year', ascending=False)['Entity'].drop_duplicates().index
income = df_income.loc[index, ['Year', 'Entity', 'Share of population below $1 a day']].sort_values(by='Entity')

df = df.join(income.set_index('Entity'), on='Country or area')
df['Share of population below $1 a day'] = df['Share of population below $1 a day'] / 100
df.fillna(0, inplace=True)

In [24]:
# Drop and rename columns for clarity
labels = ['Index', 'Country Code', 'City Code', 'Note', '2018', '2019', '2021', '2022', 'Year']
df.drop(labels=labels, inplace=True, axis=1)
df.rename(columns={'2020': 'Population (in thousands)',
                   'SpatialDimValueCode': 'Country Code',
                   'Share of population below $1 a day': 'proportion below $1 per day'
                  }, inplace=True)

In [25]:
df

,Country or area,Urban Agglomeration,Latitude,Longitude,Population (in thousands),Country Code,proportion dirty fuel,estimated population burning dirty fuels,proportion electricity access,proportion below $1 per day
0,Afghanistan,Herat,34.3482,62.1997,606,AFG,0.163,98778.0,0.995000,0.000000
1,Afghanistan,Kabul,34.5289,69.1725,4222,AFG,0.163,688186.0,0.995000,0.000000
2,Afghanistan,Kandahar,31.6133,65.7101,498,AFG,0.163,81174.0,0.995000,0.000000
3,Afghanistan,Mazar-e Sharif,36.7090,67.1109,533,AFG,0.163,86879.0,0.995000,0.000000
4,Albania,Tiranë (Tirana),41.3275,19.8189,494,ALB,0.071,35074.0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1855,Zambia,Lusaka,-15.4134,28.2771,2774,ZMB,0.786,2180364.0,0.823826,0.343309
1856,Zambia,Ndola,-12.9587,28.6366,542,ZMB,0.786,426012.0,0.823826,0.343309
1857,Zimbabwe,Bulawayo,-20.1500,28.5833,638,ZWE,0.202,128876.0,0.857155,0.073591
1858,Zimbabwe,Chitungwiza,-18.0127,31.0756,386,ZWE,0.202,77972.0,0.857155,0.073591


In [7]:
len(df)

1463

In [8]:
len(df['Country or area'].drop_duplicates())

121

We end up with 1463 urban agglomerations representing 121 countries.

In [26]:
df.head()

,Country or area,Urban Agglomeration,Latitude,Longitude,Population (in thousands),Country Code,proportion dirty fuel,estimated population burning dirty fuels,proportion electricity access,proportion below $1 per day
0,Afghanistan,Herat,34.3482,62.1997,606,AFG,0.163,98778.0,0.995,0.0
1,Afghanistan,Kabul,34.5289,69.1725,4222,AFG,0.163,688186.0,0.995,0.0
2,Afghanistan,Kandahar,31.6133,65.7101,498,AFG,0.163,81174.0,0.995,0.0
3,Afghanistan,Mazar-e Sharif,36.7090,67.1109,533,AFG,0.163,86879.0,0.995,0.0
4,Albania,Tiranë (Tirana),41.3275,19.8189,494,ALB,0.071,35074.0,1.000,0.0


In [28]:
pd.set_option('display.max_rows', 1500)
df.sort_values(by='estimated population burning dirty fuels', ascending=False)

,Country or area,Urban Agglomeration,Latitude,Longitude,Population (in thousands),Country Code,proportion dirty fuel,estimated population burning dirty fuels,proportion electricity access,proportion below $1 per day
661,Democratic Republic of the Congo,Kinshasa,-4.3276,15.3136,14342,COD,0.8975,12871945.0,0.406599,0.293478
1259,Nigeria,Lagos,6.4531,3.3958,14368,NGA,0.6680,9597824.0,0.839000,0.035116
60,Bangladesh,Dhaka,23.7104,90.4074,21006,BGD,0.4170,8759502.0,0.978000,0.001826
1664,United Republic of Tanzania,Dar es Salaam,-6.8235,39.2695,6702,TZA,0.8730,5850846.0,0.729000,0.052216
1347,Philippines,Manila,14.6042,120.9822,13923,PHL,0.2900,4037670.0,0.982341,0.000475
696,Ethiopia,Addis Ababa,9.0250,38.7469,4794,ETH,0.6910,3312654.0,0.932479,0.033740
1122,Madagascar,Antananarivo,-18.9137,47.5361,3369,MDG,0.9670,3257823.0,0.701620,0.431757
1614,Uganda,Kampala,0.3163,32.5822,3298,UGA,0.9745,3213901.0,0.698710,0.073315
448,China,Shanghai,31.2222,121.4581,27058,CHN,0.1020,2759916.0,1.000000,0.000000
1256,Nigeria,Kano,12.0001,8.5167,3999,NGA,0.6680,2671332.0,0.839000,0.035116


#### Compute initial addressable population as dirty fuel population times proportion access to electricity and proportion above one dollar per day

In [38]:
pd.set_option('display.max_rows', 300)
df['estimated addressable population'] = np.round(df['estimated population burning dirty fuels'] \
                                            * df['proportion electricity access'] \
                                            * (1 - df['proportion below $1 per day'])).astype(int)

In [39]:
df = df.sort_values(by='estimated addressable population', ascending=False).reset_index(drop=True)

In [42]:
12871945 * 0.406599

5233719.965055

In [40]:
df

,Country or area,Urban Agglomeration,Latitude,Longitude,Population (in thousands),Country Code,proportion dirty fuel,estimated population burning dirty fuels,proportion electricity access,proportion below $1 per day,estimated addressable population
0,Bangladesh,Dhaka,23.7104,90.4074,21006,BGD,0.4170,8759502.0,0.978000,0.001826,8551148
1,Nigeria,Lagos,6.4531,3.3958,14368,NGA,0.6680,9597824.0,0.839000,0.035116,7769797
2,United Republic of Tanzania,Dar es Salaam,-6.8235,39.2695,6702,TZA,0.8730,5850846.0,0.729000,0.052216,4042550
3,Philippines,Manila,14.6042,120.9822,13923,PHL,0.2900,4037670.0,0.982341,0.000475,3964484
4,Democratic Republic of the Congo,Kinshasa,-4.3276,15.3136,14342,COD,0.8975,12871945.0,0.406599,0.293478,3697742
...,...,...,...,...,...,...,...,...,...,...,...
1458,Jordan,Al-Quwaysimah,31.9167,35.9500,385,JOR,0.0000,0.0,1.000000,0.000000,0
1459,Egypt,Al-Mansurah,31.0364,31.3807,556,EGY,0.0000,0.0,1.000000,0.000123,0
1460,Jordan,Zarqa,32.0683,36.0889,729,JOR,0.0000,0.0,1.000000,0.000000,0
1461,Argentina,Posadas,-27.3671,-55.8961,389,ARG,0.0000,0.0,1.000000,0.002421,0


In [45]:
df.to_csv('../Data/addressable-population-estimates.csv', index=False)

# Visuals

In [52]:
df.groupby('Country or area')['estimated addressable population'].sum().sort_values(ascending=False)

Country or area
China                                 59822592
Nigeria                               31141913
India                                 22200713
Bangladesh                            13000952
Philippines                            8780065
Democratic Republic of the Congo       7433900
Pakistan                               6887215
United Republic of Tanzania            6241765
Mexico                                 6236186
Spain                                  5832900
Cameroon                               5347217
Ghana                                  4242758
Ethiopia                               4025750
Russian Federation                     3823235
Kenya                                  3529598
Viet Nam                               3273729
Indonesia                              3026559
Mali                                   2731611
Mozambique                             2566212
Thailand                               2505804
Somalia                                24005

In [48]:
px.bar(df[:25], x='Urban Agglomeration', y='estimated addressable population')

In [57]:
df.head()

,Country or area,Urban Agglomeration,Latitude,Longitude,Population (in thousands),Country Code,proportion dirty fuel,estimated population burning dirty fuels,proportion electricity access,proportion below $1 per day,estimated addressable population
0,Bangladesh,Dhaka,23.7104,90.4074,21006,BGD,0.4170,8759502.0,0.978000,0.001826,8551148
1,Nigeria,Lagos,6.4531,3.3958,14368,NGA,0.6680,9597824.0,0.839000,0.035116,7769797
2,United Republic of Tanzania,Dar es Salaam,-6.8235,39.2695,6702,TZA,0.8730,5850846.0,0.729000,0.052216,4042550
3,Philippines,Manila,14.6042,120.9822,13923,PHL,0.2900,4037670.0,0.982341,0.000475,3964484
4,Democratic Republic of the Congo,Kinshasa,-4.3276,15.3136,14342,COD,0.8975,12871945.0,0.406599,0.293478,3697742


In [59]:
px.scatter_geo(df, lat='Latitude', lon='Longitude', hover_name='Urban Agglomeration', size='estimated addressable population')